## Populate Task

In [1]:
import sys
from pathlib import Path

# Đường dẫn tới thư mục gốc của dự án
project_root = Path('C:/Users/ADMIN/Desktop/DATN/Extract_Information')
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

In [2]:
task = 'generated_knowledge'
model_name = 'GPT'

In [3]:
PATH_FILE_TASK = 'C:/Users/ADMIN/Desktop/DATN/Extract_Information/tasks/task_template/'
PATH_FILE_TEMPLATE = 'C:/Users/ADMIN/Desktop/DATN/Extract_Information/tasks/tasks/task_template.json'
#PATH_FILE_DATA = 'C:/Users/ADMIN/Desktop/DATN/Extract_Information/data/mave_filtered_test.jsonl'
PATH_FILE_DATA = 'C:/Users/ADMIN/Desktop/DATN/Extract_Information/shuffle/output/permuted_8.jsonl'

if model_name == 'GPT':
    #PATH_FILE_RESULT = 'C:/Users/ADMIN/Desktop/DATN/Extract_Information/tasks/result/GPT'
    PATH_FILE_RESULT = 'C:/Users/ADMIN/Desktop/DATN/Extract_Information/shuffle/results/Generated_Knowledge/'
else:
    PATH_FILE_RESULT = 'C:/Users/ADMIN/Desktop/DATN/Extract_Information/tasks/result/Llama'

In [4]:
from dotenv import load_dotenv

# Load OPENAI_API_KEY from .env file
load_dotenv()

True

In [7]:
import json
from pieutils.pieutils import update_task_dict_from_file, combine_example, calculate_recall_precision_f1

# Load task template
with open(PATH_FILE_TEMPLATE, 'r') as f:
    task_dict = json.load(f)

task_dict['task_prefix'] = "Extract all attribute-value pairs from the product title. Afterwards, " \
                            "return only the exact attribute value of the requested attribute. Your " \
                            "answer must only contain the attribute value. If the attribute is not " \
                            "applicable or the attribute value cannot be derived from the product title, " \
                            "respond with 'I do not know.'."
task_dict['shots'] = 0

# Generate examples
task_dict = update_task_dict_from_file(PATH_FILE_DATA, task_dict, True)

with open(PATH_FILE_TASK + 'task_{}.json'.format(task), 'w', encoding='utf-8') as f:
    json.dump(task_dict, f, indent=4)

## Evaluate Task

In [6]:
# Initialize processing
from tqdm import tqdm
import torch
import transformers
from langchain import LLMChain, HuggingFacePipeline
from langchain.chat_models import ChatOpenAI

from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.callbacks import get_openai_callback

In [7]:
# Model
if model_name == 'GPT':
    llm = ChatOpenAI(model_name='gpt-3.5-turbo-0125', temperature=0)
else:
    tokenizer = AutoTokenizer.from_pretrained('meta-llama/Meta-Llama-3-8B', cache_dir="/ceph/alebrink/cache")
    model = AutoModelForCausalLM.from_pretrained(
        'meta-llama/Meta-Llama-3-8B',
        cache_dir="/ceph/alebrink/cache",
        torch_dtype=torch.float32,
        device_map="auto",
        offload_folder="save_folder",
    )
    model.tie_weights()

    hf_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        device_map="auto"
    )

    terminators = [
        hf_pipeline.tokenizer.eos_token_id,
        hf_pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    llm = HuggingFacePipeline(pipeline=hf_pipeline)

In [8]:
# Put task prefix into system message.
prompt_list = []
system_message_prompt = SystemMessagePromptTemplate.from_template(task_dict['task_prefix'])
prompt_list.append(system_message_prompt)

# Add 
human_template="{input_string}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
prompt_list.append(human_message_prompt)

chat_prompt = ChatPromptTemplate.from_messages(prompt_list)

llm_chain = LLMChain(
    prompt=chat_prompt,
    llm=llm
)

# Save model
task_dict['model'] = model_name

In [9]:
def select_and_run_llm(attribute, input):
        pred = None
        if len(input) == 0:
            # No input text provided.
            return pred
        
        if model_name == 'GPT':
            response = llm_chain.run(input_string = input)
        else:
            messages = [{"role": "system", "content": chat_prompt.messages[0].format().content},
                        {"role": "human", "content": chat_prompt.messages[1].format(input_string = input).content}]

            hf_prompt = hf_pipeline.tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                add_generation_prompt=True
            )

            hf_outputs = hf_pipeline(hf_prompt, max_new_tokens=256,
                                    eos_token_id=terminators,
                                    do_sample=True,
                                    temperature=0,
                                    top_p=0.9
                                    )
            response = hf_outputs[0]["generated_text"][len(hf_prompt):]    
            
        try:
            pred ={}
            pred[attribute] = response
            print(response)
        except Exception as e:
            print(e)
            print('Response: ')
            print(response)
        return pred

In [10]:
if model_name == 'GPT':
    with get_openai_callback() as cb:
        preds = [select_and_run_llm(example['attribute'], example['input']) for example in tqdm(task_dict['examples'])]
    
    task_dict['total_tokens'] = cb.total_tokens
    print(f"Total Tokens: {cb.total_tokens}")
else:
    preds = [select_and_run_llm(example['attribute'], example['input']) for example in tqdm(task_dict['examples'])]

  0%|          | 1/562 [00:01<13:30,  1.44s/it]

12.1"


  0%|          | 2/562 [00:01<08:33,  1.09it/s]

15.4"


  1%|          | 3/562 [00:02<07:41,  1.21it/s]

...

2GB


100%|█████████▉| 561/562 [05:35<00:00,  1.84it/s]

32GB


100%|██████████| 562/562 [05:36<00:00,  1.67it/s]

8GB
Total Tokens: 66698


In [11]:
print(preds)

[{'Screen Size': '12.1"'}, {'Screen Size': '15.4"'}, {'Screen Size': '15.4"'}, {'Screen Size': '17"'}, {'Screen Size': '13.3"'}, {'Screen Size': '14.1"'}, {'Screen Size': '15.4"'}, {'Screen Size': '15.4"'}, {'Screen Size': '17"'}, {'Screen Size': '15.4-Inch'}, {'Screen Size': '17"'}, {'Screen Size': '17"'}, {'Processor Speed': '1.66 GHz'}, {'Screen Size': '15.4"'}, {'Processor Speed': '1.6 GHz'}, {'Processor Speed': '2.16 GHz'}, {'Screen Size': '13.3-inch'}, {'Screen Size': '13.3"'}, {'Processor Speed': '2 GHz'}, {'Processor Speed': '1.60GHz'}, {'Processor Speed': '2.53GHz'}, {'Processor Speed': '1.33 GHz'}, {'Processor Speed': '1.8 GHz'}, {'Processor Speed': '2.2 GHz'}, {'Number of Cores': 'Dual-Core'}, {'Number of Cores': 'Dual Core'}, {'Number of Cores': 'Dual-Core'}, {'Processor Speed': '1.9 GHz'}, {'Number of Cores': 'Dual Core'}, {'Processor Speed': '1.83GHz'}, {'Processor Speed': '1.86GHz'}, {'Sensor Type': 'CCD'}, {'Number of Cores': 'Dual Core'}, {'Processor Speed': '1.83 GHz'

In [12]:
# result_file = PATH_FILE_RESULT + 'task_run_chat_{}_{}.json'.format(task, 'gpt-3.5-turbo-0125')
# print(result_file)
# with open(result_file, 'r') as f:
#     task_dict = json.load(f)

targets = [example['target_scores'] for example in task_dict['examples']]
categories = [example['category'] for example in task_dict['examples']]
attributes = [example['attribute'] for example in task_dict['examples']]  

postprocessed_preds = [pred[attribute] for pred, attribute in zip(preds, attributes)]
#postprocessed_preds = [example['post_pred'] for example in task_dict['examples']]

task_dict['examples'] = [combine_example(example, pred, post_pred) 
                    for example, pred, post_pred in zip(task_dict['examples'], preds, postprocessed_preds)]

#print(task_dict['examples'])

results = calculate_recall_precision_f1(targets, postprocessed_preds, categories, attributes)

Attribute: Processor Speed - Category: Laptops
{'precision': 89.13, 'recall': 93.18, 'f1': 91.11}
Attribute: Processor Brand - Category: Laptops
{'precision': 89.58, 'recall': 97.73, 'f1': 93.48}
Attribute: Number of Cores - Category: Laptops
{'precision': 100.0, 'recall': 100.0, 'f1': 100.0}
Attribute: Screen Size - Category: Laptops
{'precision': 97.56, 'recall': 97.56, 'f1': 97.56}
Attribute: Battery Life - Category: Laptops
{'precision': 85.71, 'recall': 85.71, 'f1': 85.71}
Attribute: Resolution - Category: Laptops
{'precision': 100.0, 'recall': 100.0, 'f1': 100.0}
Attribute: Capacity - Category: Flash Memory Cards
{'precision': 100.0, 'recall': 100.0, 'f1': 100.0}
Attribute: SD Format - Category: Flash Memory Cards
{'precision': 75.76, 'recall': 60.98, 'f1': 67.57}
Attribute: Sensor Type - Category: Digital Cameras
{'precision': 97.5, 'recall': 97.5, 'f1': 97.5}
Attribute: Optical Zoom - Category: Digital Cameras
{'precision': 95.12, 'recall': 97.5, 'f1': 96.3}
Attribute: Sensor S

In [13]:
# Save populated task
result_file = PATH_FILE_RESULT + '{}_{}_9.json'.format(task, task_dict['model'])
with open(result_file, 'w') as fp:
    json.dump(task_dict, fp, indent=4)

In [14]:
# Error Analysis
print('Prompts for which target and postprocessed prediction do not match.')
print('-----------')
input_texts = [example['input'] for example in task_dict['examples']]

for input_text, target, pred, post_pred in zip(input_texts, targets, preds, postprocessed_preds):
    if post_pred not in target.keys():
        #print('Prompt: {}'.format(chat_prompt.format(task_prefix=task_dict['task_prefix'], input_string= input_text, human_msg_0= human_text_0, ai_msg=system_text,)))
        print('Input: {}'.format(input_text))
        print('Prediction: \n {}'.format(pred))
        print('Prediction 2: {}'.format(post_pred))
        print('Target: {}'.format(target))
        print('-----------')

Prompts for which target and postprocessed prediction do not match.
-----------
Input: Lenovo Ideapad U-350 13.3-Inch Black Laptop - Up to 5 Hours of Battery Life (Windows 7 Home Premium) 
 Requested Attribute: Battery Life
Prediction: 
 {'Battery Life': 'Up to 5 Hours'}
Prediction 2: Up to 5 Hours
Target: {'5 Hours': 1}
-----------
Input: HP Chromebook 13 G1, 13.3" (W0T01UT#ABA) 
 Requested Attribute: Processor Brand
Prediction: 
 {'Processor Brand': 'Intel'}
Prediction 2: Intel
Target: {'I do not know.': 1}
-----------
Input: Nikon COOLPIX L840 16MP 38x Opt Zoom Digital Camera 8GB Accessory Bundle - Black - Includes Camera, 8GB Secure Digital SD Memory Card, Compact Deluxe Gadget Bag, Cleaning Kit and Cleaning Cloth 
 Requested Attribute: Optical Zoom
Prediction: 
 {'Optical Zoom': '38x Opt Zoom'}
Prediction 2: 38x Opt Zoom
Target: {'38x': 1}
-----------
Input: SanDisk Ultra 32GB UHSI Class 10 80mb/s MicroSDHC Memory Card for Samsung Galaxy E7 E5 A5 A3 V S5Plus Grand Core Prime Tab A